In [ ]:
#This notebook is to make growth curves from selected data output from the Biotek using python's Seaborn and Altair packages
#Seaborn and Altair are packages that allow you to make nice figures
#This code only works for excel files with well definitions in row 1, and time column A


# remove hashes in front of pip code and use it to install altair, 
# if you dont have other packages change 'altair' to the package you wish to install
#the final cell in this notebook makes a chart using Altair. Altair makes them pretty nicely
#Altair also appears to be faster than seaborn
#if altair doesnt install with the above command, there is also seaborn code (baked into anaconda) to make your figure

In [ ]:
#import pip
#pip.main(['install', 'altair']) 

In [1]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
%pylab inline
import seaborn as sns
import pandas as pd
import math as mt
import altair as alt

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
dfOD=0
excel='DBTL5.xlsx'
dfOD=pd.read_excel(excel,sheet_name='od')
dfPH=pd.read_excel(excel, sheet_name='ph') 
dfDO=pd.read_excel(excel, sheet_name='do')
dic=pd.read_excel(excel, sheet_name='dic')

columns = dfOD.columns.to_list()
columns=columns[1:]

dfOD = dfOD.melt(id_vars='Time', value_vars=columns)
dfPH = dfPH.melt(id_vars='Time', value_vars=columns)
dfDO = dfDO.melt(id_vars='Time',value_vars=columns)

dfOD=dfOD.rename(columns={'value':'OD600','variable':'well'})
dfOD['pH']=dfPH.value.values
dfOD['DO']=dfDO.value.values


dfOD['condition'] = dfOD.well.values 
dfOD=dfOD.replace({'condition':dict(zip(dic.Well1,dic.Condition))}) 
dfOD=dfOD.replace({'condition':dict(zip(dic.Well2,dic.Condition))})
dfOD=dfOD.replace({'condition':dict(zip(dic.Well3,dic.Condition))})
#dfOD=dfOD.replace({'condition':dict(zip(dic.Well4,dic.Condition))})


#altair cant have more than 5 lines per line chart. 
#we can make more and overlay them, that is what the conditions and various "dfOD"s are for


condition1 = ("A1C1","A2C2","A3C3","A4C4","A8C8")
condition2 = ("D4F4","D1F1","D2F2","D3F3","A7C7")            
condition3 = ("A6C6","D5F5",'D6F6','D7F7','MOPS')

dfOD0=dfOD[dfOD.condition.isin((condition1))]
dfOD1=dfOD[dfOD.condition.isin((condition2))]
dfOD2=dfOD[dfOD.condition.isin((condition3))]

from colour import Color
classlen=int(len(set(dfOD.condition)))
grey = Color('#c2c0c0')
red = Color('#ff001e')
green = Color('#006918')
colg=int(math.trunc(classlen/3))
colb=int(classlen-2*colg)


colors = list(grey.range_to(Color('#000000'),colb))
colorsc =list(green.range_to(Color('#ffd900'),colg)) #toggle these based on classlen
colorsb= list(red.range_to(Color("#0d00ff"),colg))
             
colors = [str(i) for i in colors]
colorsb= [str(i) for i in colorsb]
colorsc=[str(i) for i in colorsc]
             
stroke_palette = colorsb +colorsc+ colors 
#sns.palplot(stroke_palette)

#a semi-interactive chart containing error bars
def lecterr(df,y ,ytitle, y_max,y_min ):
    line = alt.Chart(df).mark_line(opacity=1).encode(
         x=alt.X('Time',axis=alt.Axis(labels=True),title=None,scale=alt.Scale(domain=(0, 48))
                ),
         y=alt.Y(y,axis=alt.Axis(labels=True),title=ytitle,
                 scale=alt.Scale(domain=(y_min,y_max),clamp=True)
                ),
         color=alt.Color('condition', legend=alt.Legend(columns=1,symbolLimit=0,orient='left'), 
         scale=alt.Scale(range=stroke_palette)))
    error= alt.Chart(df).mark_errorband(extent='ci').encode(
         x=alt.X('Time',axis=alt.Axis(labels=True),title=None,scale=alt.Scale(domain=(0, 48))
                ),
         y=alt.Y(y,axis=alt.Axis(labels=True),title=ytitle,
                 scale=alt.Scale(domain=(y_min,y_max),clamp=True)
                ),
         color=alt.Color('condition', legend=alt.Legend(columns=1,symbolLimit=0,orient='left'), 
         scale=alt.Scale(range=stroke_palette)))
    chart = line + error
    return chart
def lecterrfig(y,ytitle,y_max,y_min):   
    line2 = lecterr(df=dfOD0,y=y,ytitle=ytitle,y_max=y_max,y_min=y_min)
    line3 = lecterr(df=dfOD1,y=y,ytitle=ytitle,y_max=y_max,y_min=y_min)
    line4 = lecterr(df=dfOD2,y=y,ytitle=ytitle,y_max=y_max,y_min=y_min)
    chart=line2+line3+line4
    #.resolve_scale(y='independent')
    return chart

#a figure letting you select each condition
def lectori(df,y ,ytitle, y_max,y_min ):
    selector=alt.selection_multi(fields=['condition'],bind='legend')
    selector2=alt.selection_multi()
    line = alt.Chart(df).mark_line(opacity=1).encode(
         x=alt.X('Time',axis=alt.Axis(labels=True),title=None,scale=alt.Scale(domain=(0, 48))
                ),
         y=alt.Y(y,axis=alt.Axis(labels=True),title=ytitle,
                 scale=alt.Scale(domain=(y_min,y_max),clamp=True)
                ),
         tooltip=['condition'],
         opacity=alt.condition(selector|selector2, 
                           alt.value(1.), 
                           alt.value(0.25)),
         color=alt.condition(selector|selector2,alt.Color('condition:N', legend=alt.Legend(columns=1,symbolLimit=0,orient='left',title='formula',symbolType='circle'), 
         scale=alt.Scale(range=stroke_palette)),alt.value('gray'))).add_selection(selector,selector2)
    chart=line
    return chart
    
def lectorfigi(y,ytitle,y_max,y_min):   
    line2 = lectori(df=dfOD0,y=y,ytitle=ytitle,y_max=y_max,y_min=y_min)
    line3 = lectori(df=dfOD1,y=y,ytitle=ytitle,y_max=y_max,y_min=y_min)
    line4 = lectori(df=dfOD2,y=y,ytitle=ytitle,y_max=y_max,y_min=y_min)
    chart=line2+line3+line4
    #.resolve_scale(y='independent')
    return chart

In [5]:
od=lecterrfig('mean(OD600)',"\u03BB620 Backscatter (a.u.)",20,0)
ph=lecterrfig('mean(pH)',"pH",8,0)
DO=lecterrfig('mean(DO)',"DO",120,-20)
chart1=od|ph|DO
chart1=chart1.configure_axis(
        labelFontSize=20,
        titleFontSize=20)
    # ).configure_legend(
    #     labelFontSize=20,
    #     titleFontSize=20,
    #     symbolSize=150)
chart1#.save('DBTL5.html')

alt.HConcatChart(...)

In [6]:
od=lectorfigi('mean(OD600)',"\u03BB620 Backscatter (a.u.)",20,0)
ph=lectorfigi('mean(pH)',"pH",8,0)
DO=lectorfigi('mean(DO)',"DO",120,-20)
chart2=od|ph|DO
chart2=chart2.configure_axis(
        labelFontSize=20,
        titleFontSize=20
    ).configure_legend(
        labelFontSize=20,
        titleFontSize=20,
        symbolSize=150)
chart2

alt.HConcatChart(...)